In [ ]:
# import sys
import pathlib
# import os

# # in order to do relative import
# sys.path.append(str(pathlib.Path(os.getcwd()).parent))
# sys.path.append(str(pathlib.Path(os.getcwd()).parent.parent))

In [ ]:
import numpy as np

from residual_actions import ResidualActionsSettings, ResidualActionsLearner

# Read data

In [ ]:
with open('datasets/expdata_1_2024_12_03_dodge_and_collect_four_min_4800.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True)

In [ ]:
SETTINGS = ResidualActionsSettings(
    history_size=40,
    hidden_channels_memory=128,
    hidden_channels_behavior=128,
    batch_size=256,
    optim_learning_rate=0.001,
    target_loss=1e-07,
    force_stop_at_plateau_epochs=20,
    train_log_frequency=5,
    running_loss_window_size=10,
    min_epochs=100,
    grace_epochs_after_min_epochs=20,
    max_epochs=900
)

In [ ]:
learner = ResidualActionsLearner(
    state_space_size=
    action_space_size=
    settings=SETTINGS,
    device='cuda'
)